In [40]:
from transformers import pipeline

1. Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.
1. Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository. 

In [41]:
models = [
    ('bert-base-multilingual-cased', '[MASK]'),
    ('xlm-roberta-base', '<mask>'),
    ('Geotrend/distilbert-base-pl-cased', '[MASK]')
]

unmaskers = list(
    map(lambda model, mask: (model, pipeline('fill-mask', model=model), mask), *zip(*models))
)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/141k [00:00<?, ?B/s]

In [42]:
MASK = '[[mask]]'

def test_sentences(sentences: list[str]):
    for sentence in sentences:  
        print('#'*20)
        for name, unmasker, mask in unmaskers:
            sentence_m = sentence.replace(MASK,mask)
            print(f'\033[1m{name}\033[0m: ', end='')
            for result in unmasker(sentence_m):
                print(f'{result["sequence"]}', end=' # ')
            print()

3. Devise a method to test if the langage model understands Polish cases. E.g. testing for *nominal case* could be expressed as "Warszawa to największe `[MASK]`", and the masked word should be in nominative case. Create sentences for each case.

In [43]:
sentences = [
    f'{MASK} jest popularnym owocem', # mianownik
    f'Nie ma tutaj z nami {MASK}', # dopełniacz
    f'Przyglądam się {MASK} od godziny', # celownik
    f'Czy widziałeś ostatnio {MASK}', # biernik
    f'Obszar gnuśności zalany {MASK}', # narzędnik
    f'Rozmawialiśmy z nimi o {MASK}', # miejscownik
    f'{MASK}! dodaj mi skrzydła', # wołacz
]

sentences_adj = [
    f'Czerwony {MASK} jest popularnym owocem', # mianownik
    f'Nie ma tutaj z nami naszego {MASK}', # dopełniacz
    f'Przyglądam się pięknemu {MASK} od godziny', # celownik
    f'Czy widziałeś ostatnio niebieski {MASK}', # biernik
    f'Poszedłem na spacer z moim {MASK}', # narzędnik
    f'Rozmawialiśmy z nimi o pięknych {MASK}', # miejscownik
    f'{MASK}! dodaj mi skrzydła! Niech nad martwym wzlecę światem', # wołacz
]

In [44]:
test_sentences(sentences)

####################
bert-base-multilingual-cased: jest jest popularnym owocem # nie jest popularnym owocem # - jest popularnym owocem # ta jest popularnym owocem # , jest popularnym owocem # 
xlm-roberta-base: Nie jest popularnym owocem # nie jest popularnym owocem # Dlaczego jest popularnym owocem # To jest popularnym owocem # który jest popularnym owocem # 
Geotrend/distilbert-base-pl-cased: Herb jest popularnym owocem # Jak jest popularnym owocem # Obecnie jest popularnym owocem # Nie jest popularnym owocem # Miasto jest popularnym owocem # 
####################
bert-base-multilingual-cased: Nie ma tutaj z nami. # Nie ma tutaj z nami, # Nie ma tutaj z nami ; # Nie ma tutaj z nami : # Nie ma tutaj z nami? # 
xlm-roberta-base: Nie ma tutaj z nami... # Nie ma tutaj z nami. # Nie ma tutaj z nami # Nie ma tutaj z nami! # Nie ma tutaj z nami? # 
Geotrend/distilbert-base-pl-cased: Nie ma tutaj z nami? # Nie ma tutaj z nami! # Nie ma tutaj z nami nie # Nie ma tutaj z nami : # Nie ma tutaj 

In [52]:
test_sentences(sentences_adj)

####################
bert-base-multilingual-cased: Czerwony herb jest popularnym owocem # Czerwony most jest popularnym owocem # Czerwony list jest popularnym owocem # Czerwony pan jest popularnym owocem # Czerwony Herb jest popularnym owocem # 
xlm-roberta-base: Czerwony pieprz jest popularnym owocem # Czerwony róż jest popularnym owocem # Czerwony sok jest popularnym owocem # Czerwony pomidor jest popularnym owocem # Czerwony granat jest popularnym owocem # 
Geotrend/distilbert-base-pl-cased: Czerwony gaz jest popularnym owocem # Czerwony Herb jest popularnym owocem # Czerwony Jerzy jest popularnym owocem # Czerwony piłkarz jest popularnym owocem # Czerwony czas jest popularnym owocem # 
####################
bert-base-multilingual-cased: Nie ma tutaj z nami naszego. # Nie ma tutaj z nami naszego, # Nie ma tutaj z nami naszego : # Nie ma tutaj z nami naszego - # Nie ma tutaj z nami naszego ; # 
xlm-roberta-base: Nie ma tutaj z nami naszego domu # Nie ma tutaj z nami naszego serca # Ni

4. Devise a method to test long-range relationships such as gender. E.e. you can use two verbs where withe masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [46]:
sentences = [
    f'Poszedłem do pracy i {MASK} tam cały dzień',
    f'Zobaczyła go z daleka i {MASK} za nim',
    f'{MASK} tam i zobacz co się dzieje'
]

sentences_dot = [
    f'Poszedłem do pracy i {MASK} tam cały dzień.',
    f'Zobaczyła go z daleka i {MASK} za nim.',
    f'{MASK} tam i zobacz co się dzieje.'
]

In [47]:
test_sentences(sentences)

####################
bert-base-multilingual-cased: Poszedłem do pracy i to tam cały dzień # Poszedłem do pracy i jest tam cały dzień # Poszedłem do pracy i tam tam cały dzień # Poszedłem do pracy i gra tam cały dzień # Poszedłem do pracy i ma tam cały dzień # 
xlm-roberta-base: Poszedłem do pracy i był tam cały dzień # Poszedłem do pracy i miałem tam cały dzień # Poszedłem do pracy i jestem tam cały dzień # Poszedłem do pracy i będę tam cały dzień # Poszedłem do pracy i pracował tam cały dzień # 
Geotrend/distilbert-base-pl-cased: Poszedłem do pracy i było tam cały dzień # Poszedłem do pracy i pracy tam cały dzień # Poszedłem do pracy i pracował tam cały dzień # Poszedłem do pracy i przez tam cały dzień # Poszedłem do pracy i jeszcze tam cały dzień # 
####################
bert-base-multilingual-cased: Zobaczyła go z daleka i była za nim # Zobaczyła go z daleka i to za nim # Zobaczyła go z daleka i miała za nim # Zobaczyła go z daleka i tylko za nim # Zobaczyła go z daleka i ta za nim #

In [48]:
test_sentences(sentences_dot)

####################
bert-base-multilingual-cased: Poszedłem do pracy i to tam cały dzień. # Poszedłem do pracy i jest tam cały dzień. # Poszedłem do pracy i było tam cały dzień. # Poszedłem do pracy i pracy tam cały dzień. # Poszedłem do pracy i tym tam cały dzień. # 
xlm-roberta-base: Poszedłem do pracy i był tam cały dzień. # Poszedłem do pracy i miałem tam cały dzień. # Poszedłem do pracy i pracował tam cały dzień. # Poszedłem do pracy i jestem tam cały dzień. # Poszedłem do pracy i stał tam cały dzień. # 
Geotrend/distilbert-base-pl-cased: Poszedłem do pracy i pracował tam cały dzień. # Poszedłem do pracy i było tam cały dzień. # Poszedłem do pracy i przez tam cały dzień. # Poszedłem do pracy i zmarł tam cały dzień. # Poszedłem do pracy i pracy tam cały dzień. # 
####################
bert-base-multilingual-cased: Zobaczyła go z daleka i była za nim. # Zobaczyła go z daleka i to za nim. # Zobaczyła go z daleka i została za nim. # Zobaczyła go z daleka i miała za nim. # Zobaczyła go

5. Check if the model captures real-world knolwedge. For instance a sentence "`[MASK]` wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [49]:
sentences = [
    f'Ziemia to {MASK} planeta od Słońca',
    f'Najwyższą górą na Ziemi jest {MASK}',
    f'Kod genetyczny komórki przechowywany jest w jej {MASK}'
]

sentences_long = [
    f'Ziemia, nazywana niebieską planetą, to {MASK} planeta od Słońca',
    f'Najwyższą górą na Ziemi jest {MASK} o wysokości 8848 m n.p.m.',
    f'Kod genetyczny komórki przechowywany jest w jej {MASK} w postaci łańcuchów DNA'
]

In [50]:
test_sentences(sentences)

####################
bert-base-multilingual-cased: Ziemia to druga planeta od Słońca # Ziemia to to planeta od Słońca # Ziemia to ta planeta od Słońca # Ziemia to jedna planeta od Słońca # Ziemia to od planeta od Słońca # 
xlm-roberta-base: Ziemia to druga planeta od Słońca # Ziemia to większa planeta od Słońca # Ziemia to jedna planeta od Słońca # Ziemia to mniej planeta od Słońca # Ziemia to bliżej planeta od Słońca # 
Geotrend/distilbert-base-pl-cased: Ziemia to druga planeta od Słońca # Ziemia to pierwsza planeta od Słońca # Ziemia to jedna planeta od Słońca # Ziemia to tylko planeta od Słońca # Ziemia to nazwa planeta od Słońca # 
####################
bert-base-multilingual-cased: Najwyższą górą na Ziemi jest : # Najwyższą górą na Ziemi jest. # Najwyższą górą na Ziemi jest to # Najwyższą górą na Ziemi jest góry # Najwyższą górą na Ziemi jest - # 
xlm-roberta-base: Najwyższą górą na Ziemi jest: # Najwyższą górą na Ziemi jest... # Najwyższą górą na Ziemi jest... # Najwyższą górą na 

In [51]:
test_sentences(sentences_long)

####################
bert-base-multilingual-cased: Ziemia, nazywana niebieską planetą, to jest planeta od Słońca # Ziemia, nazywana niebieską planetą, to to planeta od Słońca # Ziemia, nazywana niebieską planetą, to druga planeta od Słońca # Ziemia, nazywana niebieską planetą, to więc planeta od Słońca # Ziemia, nazywana niebieską planetą, to : planeta od Słońca # 
xlm-roberta-base: Ziemia, nazywana niebieską planetą, to druga planeta od Słońca # Ziemia, nazywana niebieską planetą, to większa planeta od Słońca # Ziemia, nazywana niebieską planetą, to pierwsza planeta od Słońca # Ziemia, nazywana niebieską planetą, to główna planeta od Słońca # Ziemia, nazywana niebieską planetą, to jedna planeta od Słońca # 
Geotrend/distilbert-base-pl-cased: Ziemia, nazywana niebieską planetą, to druga planeta od Słońca # Ziemia, nazywana niebieską planetą, to pierwsza planeta od Słońca # Ziemia, nazywana niebieską planetą, to jedna planeta od Słońca # Ziemia, nazywana niebieską planetą, to tylko plan

6. Take into accout the fact, that causal language models such as PapuGaPT2 or plT5, will only generate continuations of the sentenes, so the
   examples have to be created according to that paradigm.

7. Answer the following questions:
   1. Which of the models produced the best results?
   1. Was any of the models able to capture Polish grammar?
   1. Was any of the models able to capture long-distant relationships between the words?
   1. Was any of the models able to capture world knowledge?
   1. What are the most striking errors made by the models?